In [1]:
import os

proxy = "http://proxyout.lanl.gov:8080"

os.environ['http_proxy'] = proxy 
os.environ['HTTP_PROXY'] = proxy
os.environ['https_proxy'] = proxy
os.environ['HTTPS_PROXY'] = proxy

In [2]:
import ee
#ee.Authenticate()
ee.Initialize()
import geemap
#geemap.update_package()
import pandas as pd

ModuleNotFoundError: No module named 'ee'

In [3]:
states = ee.FeatureCollection("TIGER/2018/States")
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover')

In [4]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map.addLayer(landcover, {}, 'Land cover')
Map.addLayer(states, {}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

MACA processor

In [31]:
YearZeroStart=pd.to_datetime('2050-08-01')
YearZeroEnd= pd.to_datetime('2050-08-15')
Lon=-77.2214
Lat=37.2665



AOI=ee.Geometry.Rectangle([Lon+.05, Lat-0.5, Lon-.05, Lat-0.5])
#Create a buffer zone around a point, such


MACA = ee.ImageCollection("IDAHO_EPSCOR/MACAv2_METDATA")
#xy = ee.Geometry.Point([Lon, Lat])
sub=ee.ImageCollection(MACA.filterDate(YearZeroStart, YearZeroEnd)
                       .filterMetadata('model','equals','inmcm4')
                       .filterMetadata('scenario','equals','rcp45')
                       .filterMetadata('ensemble','equals', 'r1i1p1' ))
sub


Map = geemap.Map(center=(40, -100), zoom=4)
Map.addLayer(sub, {}, 'Land cover')
Map.addLayer(AOI, {}, 'Land cover')
#Map.addLayer(states, {}, "US States")
Map

#xt=xy.buffer(10)
#xt=geemap.shp_to_ee(xt)
#print(xt)
#value = sub.get('tasmax');
#out_csv = os.path.join('landsat.csv')
#geemap.extract_values_to_points(xy, sub, out_csv)

#var='tasmax'
#def createTS(img):
#  date = img.get('system_time_start')
#  value = img.reduceRegion(ee.Reducer.mean(), AOI).get(var)
  #ET = ee.Feature('null', {'system:time_start': date, 
   #                          'date': ee.Date(date).format('Y/M/d'), 
   #                          'value': value})
#  return(value)
#geemap.ee_to_numpy(value, region=AOI)
#first=MACA.map(createTS)
# print(first)

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [32]:
def poi_mean(img):
    mean = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=AOI).get('tasmax')
    return img.set('date', img.date().format()).set('mean',mean)

In [33]:
poi_reduced_imgs = sub.map(poi_mean)
nested_list = poi_reduced_imgs.reduceColumns(ee.Reducer.toList(2), ['date','mean']).values().get(0)
df = pd.DataFrame(nested_list.getInfo(), columns=['date','mean'])

df

,date,mean
0,2050-08-01T00:00:00,302.262144
1,2050-08-02T00:00:00,305.727214
2,2050-08-03T00:00:00,304.529856
3,2050-08-04T00:00:00,302.555328
4,2050-08-05T00:00:00,302.448929
5,2050-08-06T00:00:00,304.211644
6,2050-08-07T00:00:00,304.173151
7,2050-08-08T00:00:00,305.670582
8,2050-08-09T00:00:00,306.912835
9,2050-08-10T00:00:00,308.319903
